## Session 02: Decision Trees & Random Forests

In [1]:
from collections import Counter

import numpy as np
import pandas as pd

np.random.seed(42)

**From Wikipedia**

_Decision tree learning uses a decision tree (as a predictive model) to go from observations about an item (represented in the branches) to conclusions about the item's target value (represented in the leaves). It is one of the predictive modelling approaches used in statistics, data mining and machine learning. Tree models where the target variable can take a discrete set of values are called classification trees; in these tree structures, leaves represent class labels and branches represent conjunctions of features that lead to those class labels. Decision trees where the target variable can take continuous values (typically real numbers) are called regression trees._

We first define a very basic interface for a decision tree.

```
class DecisionTree(object):
    def __init__(self, split_scorer):
        ...

    def fit(self, data):
        ...

    def predict(self, row):
        ...
```

The `split_scorer` will be a callback which will allow us to determine which feature to split on next.

## Classification Decision Trees

Now that we have a basic API defined, we can write a few scorer functions, or metrics.

In [2]:
def split(df, feature):
    """Returns an array of non-overlapping dataframes, the union of which is the original"""
    return [x for _, x in df.groupby(feature)]

**Classification Error**

$$E_{CE_m} = 1 - max_{k}(\hat{p}_{mk})$$



where $\hat{p}_{mk}$ is the proportion of training observations in the mth region that are from Kth class.

This is basically the majority classifier, i.e. you predict the majority class at the leaf node.

This is typically used when you prune (or that's what the book says :P)

In [3]:
def classification_error(target):
    return 1 - target.value_counts().max()/len(target)

**Gini Impurity**

$$G = \sum_{k=1}^{N} p_i (1 - p_i)$$

Equivalent form:

$$G = 1 - \sum_{k=1}^{N} p_i^2$$

The Gini index is referred to as a measure of node purity—a small value indicates that a node contains predominantly observations from a single class.

In [4]:
def gini_impurity(target):
    pi = target.value_counts(normalize=True).values
    return 1 - pi.dot(pi)

def gini_impurity_scorer(df, target_label, epsilon=0.01):
    """Given a dataframe with the target label, determine the feature that gives the best split
        using the Gini impurity as a measure
    """
    target = df[target_label]
    features = df.drop(target_label, axis=1).columns
    
    gini_before = gini_impurity(df[target_label])
    
    best_feature = None
    
    for feature in features:
        df_split = split(df, feature)
        gini_after = np.mean([gini_impurity(sub_df[target_label]) for sub_df in df_split])
        if epsilon < gini_before - gini_after:
            best_feature = feature
            
    return best_feature

**Information Gain**

IG = entropy(parent) – weighted average entropy(children)

i.e.

$$IG = H(T) - H(T | a)$$

where entropy is given by: 

$H(T) = -\sum_{i = 1}^{K} p_i log_2(p_i)$

and 

$H(T | a) = \sum_{t \in T} p(t) H(t)$,

i.e. the weighted sum of the entropies of the children, where they are weighted by the number of samples in that subset.

In [5]:
def entropy(target):
    pi = np.array(target.value_counts(normalize=True).tolist())
    return -np.sum(pi*np.log2(pi))

def weighted_entropy(targets):
    n = sum(len(t) for t in targets)
    weights = np.array([len(t)/n for t in targets])
    entropies = [entropy(target) for target in targets]
    return -np.dot(weights, entropies)

def information_gain(df, target_label):
    """Given a dataframe with the target label, determine the feature that gives the best split
        using the Information Gain as a measure
    """
    target = df[target_label]
    features = df.drop(target_label, axis=1).columns
    
    parent_entropy = entropy(target)
    
    best_feature = None
    ig_max = -np.inf
    
    for feature in features:
        df_split = split(df, feature)
        targets = [df_subset[target_label] for df_subset in df_split]
        child_entropy = weighted_entropy(targets)
        info_gain = parent_entropy - child_entropy
        if info_gain > ig_max:
            ig_max = info_gain
            best_feature = feature
            
    return best_feature

Now that we have a few scorers avaliable, we can now sketch out an initial implementation of a decision tree:

In [6]:
class DecisionTree(object):
    def __init__(self, split_scorer):
        self.scorer = split_scorer
        self.tree = {}
        
    def _majority(self, data, target_label):
        return data[target_label].value_counts().idxmax()

    def _fit(self, data, target_label, max_depth, current_depth=0):
        features = data.drop(target_label, axis=1).columns.tolist()
        
        # Stopping criterion is when we have only a single class label left
        if len(set(data[target_label])) == 1: return data[target_label].iloc[0]

        # Or we don't have any features left to split on, 
        # in which case we will just predict majority:
        if not features: return self._majority(data, target_label)
        
        # OR we have hit the max depth of the tree (tuning parameter)
        if current_depth > max_depth: return self._majority(data, target_label)
        
        # Decide which is the best feature to split on using the split scorer
        best_feature = self.scorer(data, target_label)
        
        # Split on the best feature
        splits = split(data, best_feature)
        
        # Start a (sub) tree
        current_node = {best_feature: {}}

        # For every value that this feature can take, we fit a sub-tree to that subset of data 
        for val in set(data[best_feature]):
            # We drop this feature only because this is a categorical scenario
            # And we are not using binary splitting, but n-ary splitting.
            data_subset = data[data[best_feature] == val].drop(best_feature, axis=1)

            # Recursively fit trees for each split value of this feature
            current_node[best_feature][val] = self._fit(data_subset, target_label, max_depth, current_depth + 1)

        return current_node
                        
    def fit(self, data, target_label, max_depth=3):
        self.tree = self._fit(data, target_label, max_depth=max_depth)

    def predict(self, row):
        """The row should be a dict-like object, that is subsettable by the feature names used to train the model"""
        current_node = self.tree
        
        while type(current_node) != str:
            feature = list(current_node.keys())[0]
            actual_value = row[feature]
            
            current_node = current_node[feature][actual_value]
            
        return current_node

In [7]:
cars = pd.read_csv('car.csv', names=['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'evaluation'])

In [8]:
cars.head()

,buying,maint,doors,persons,lug_boot,safety,evaluation
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [9]:
cars.evaluation.unique()

array(['unacc', 'acc', 'vgood', 'good'], dtype=object)

In [10]:
dt = DecisionTree(gini_impurity_scorer)
dt.fit(cars, 'evaluation')

You can visualize the tree that was generated by printing out the decision dict. The below output should be interpreted left-to-right (as opposed to top-down in a normal visualization of a decision tree).

The values for a given feature all occur at the same indentation level, so it is somewhat easier to read.

In [11]:
from pprint import pprint
pprint(dt.tree)

{'safety': {'high': {'lug_boot': {'big': {'persons': {'2': 'unacc',
                                                      '4': {'maint': {'high': 'acc',
                                                                      'low': 'acc',
                                                                      'med': 'acc',
                                                                      'vhigh': 'acc'}},
                                                      'more': {'maint': {'high': 'acc',
                                                                         'low': 'acc',
                                                                         'med': 'acc',
                                                                         'vhigh': 'acc'}}}},
                                  'med': {'persons': {'2': 'unacc',
                                                      '4': {'doors': {'2': 'acc',
                                                                      '3': 'acc',
    

In [12]:
dt.predict({'safety': 'med', 'lug_boot': 'med', 'persons': '2'})

'unacc'

## Regression Trees

Building a Regression Tree

$\sum_{m = 1}^{|T|} \sum_{i: x_i \in \mathbb{R}_m} (y_i - \hat{y}_{R_m}))^2 + \alpha |T|$

Use k-folds cross-validation to pick $\alpha$:

For k = 1, 2...K:  
   1. Use recursive binary splitting to grow a large tree  
   2. Apply cost complexity pruning
  
Average results for each $\alpha$, pick $\alpha$ to minimize RSS.

## Advantages and Disadvantages of Decision Trees

From http://scikit-learn.org/stable/modules/tree.html

**Some advantages of decision trees are:**

1. Simple to understand and to interpret. Trees can be visualised.
2. Requires little data preparation. Other techniques often require data normalisation, dummy variables need to be created and blank values to be removed. Note however that this module does not support missing values.
3. The cost of using the tree (i.e., predicting data) is logarithmic in the number of data points used to train the tree.
4. Able to handle both numerical and categorical data. Other techniques are usually specialised in analysing datasets that have only one type of variable. See algorithms for more information.
5. Able to handle multi-output problems.
6. Uses a white box model. If a given situation is observable in a model, the explanation for the condition is easily explained by boolean logic. By contrast, in a black box model (e.g., in an artificial neural network), results may be more difficult to interpret.
7. Possible to validate a model using statistical tests. That makes it possible to account for the reliability of the model.
8. Performs well even if its assumptions are somewhat violated by the true model from which the data were generated.


**The disadvantages of decision trees include:**

1. Decision-tree learners can create over-complex trees that do not generalise the data well. This is called overfitting. Mechanisms such as pruning (not currently supported), setting the minimum number of samples required at a leaf node or setting the maximum depth of the tree are necessary to avoid this problem.
2. Decision trees can be unstable because small variations in the data might result in a completely different tree being generated. This problem is mitigated by using decision trees within an ensemble.
3. The problem of learning an optimal decision tree is known to be NP-complete under several aspects of optimality and even for simple concepts. Consequently, practical decision-tree learning algorithms are based on heuristic algorithms such as the greedy algorithm where locally optimal decisions are made at each node. Such algorithms cannot guarantee to return the globally optimal decision tree. This can be mitigated by training multiple trees in an ensemble learner, where the features and samples are randomly sampled with replacement.
4. There are concepts that are hard to learn because decision trees do not express them easily, such as XOR, parity or multiplexer problems.
5. Decision tree learners create biased trees if some classes dominate. It is therefore recommended to balance the dataset prior to fitting with the decision tree.

## Bagging / Bootstrap Aggregating

Concept: Take many training sets, build separate models using each training set, and then average the results

$$\hat{f}_{avg}(x) = \frac{1}{B} \sum_{b = 1}^{B} \hat{f}_b(x)$$

Realistically, you don't have multiple training sets, and so you create B samples from your training set instead.

**Training**

1. Take repeated samples from single training dateset
2. Train our model on the bth bootstrapped training set to get $\hat{f}_b(x)$
3. Average to get predictions

These trees are grown deep (not pruned)

**Prediction**

Take majority vote from predicted classes from B trees.

In practice, pick a large B so that the error settles down/converges (~100)

In [13]:
class BaggedDecisionTrees():
    def __init__(self, k, sample_frac):
        """Create a new bagged decision tree model, with k decision trees"""
        self.k = k
        self.sample_frac = sample_frac
        self.models = []
        
    def fit(self, data, target_label):
        for _ in range(self.k):
            sampled_data = data.sample(frac=self.sample_frac, replace=True)
            dt = DecisionTree(gini_impurity_scorer)
            dt.fit(sampled_data, target_label, max_depth=2)
            self.models.append(dt)
            
    def predict(self, row):
        predictions = [model.predict(row) for model in self.models]
        return Counter(predictions).most_common(1)[0]

In [14]:
bdt = BaggedDecisionTrees(20, 0.5)
bdt.fit(cars, 'evaluation')

In [15]:
bdt.predict({'safety': 'med', 'lug_boot': 'med', 'persons': '4', 'doors': '3'})

('acc', 12)

We've used an artificially low value for the sampling fraction, as well as the number of trees, just to illustrate that a majority vote is being taken for the prediction.

We can see that out of the 20 classifiers that we trained, 12 of them voted for 'acc', and so that is our final prediction.

## Random Forests

This is very similar to bagging, but decorrelates the trees.

You randomly sample from the set of predictions $X = {X_1, X_2, X_3 \ldots X_p}$, say $m = {X_1, X_4, X_6 \ldots}$.

The rationale is that if you have a very strong predictor $X_k$, then in regular bagging it will end up in most of the trees.

$m \simeq \sqrt{p}$ is an ideal choice.

$m = p$ is just regular bagging.

In general, small m is helpful when you have a large number of correlated predictors.